In [ ]:
import sys
from qwikidata.sparql import return_sparql_query_results
from qwikidata.entity import WikidataItem, WikidataLexeme, WikidataProperty
from qwikidata.linked_data_interface import get_entity_dict_from_api
from SPARQLWrapper import SPARQLWrapper, JSON
import json
from tqdm import tqdm
import re
import requests as r
import pandas as pd
from collections import defaultdict
from libindic import inexactsearch
import pandas as pd
import pickle as pkl
import wptools
sys.path.append('..')
from method2.Places_translator import Places_translator
translator = Places_translator()
from method1.utils import *
from baseline.baseline import baseline_infobox
from method1.get_infobox import method1_infobox

In [ ]:
with open('../data-collection/places/final_places.json') as f:
    data = json.loads(f.read())['data']
for ind , val in enumerate(data):
    val['index'] = ind

In [ ]:
%timeit
for entry in data[:1800]:
    try :
        print(entry['wd_id'] , entry['en_wikipedia_title'])
        baseline = baseline_infobox(entry['wd_id'] ,category='places')
        method2 = translator.get_infobox(entry['en_wikipedia_title'] , test=True)
        method1 = method1_infobox(entry['wd_id'],'people')
        output = {
            "method_1" : method1,
            "method_2" : method2,
            "baseline" : baseline
        }
        with open('places_till_1800.jsonl' , 'a+') as f:
            f.write(json.dumps({entry['index'] : output} , ensure_ascii = False))
            f.write("\n")
    except Exception as e:
        print(e)        